In [8]:
# define a model using pytorch
# a pretrained model adding layers to classify the 4 classes of the images
import torch
import torch.nn as nn
from torchvision import models

class MIDmodel(nn.Module):
    def __init__(self):
        super(MIDmodel, self).__init__()
        # first pretrained
        self.pretrained = models.resnet18(pretrained=True)
        # add layers
        self.fc1 = nn.Linear(1000, 512)
        self.fc3 = nn.Linear(512, 4)
       #  self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # set x to three channels
        x = x.repeat(1, 3, 1, 1)
        x = self.pretrained(x)
        # x = self.dropout(x)
        x = self.fc1(x)
        # x = self.dropout(x)
        x = self.fc3(x)
        return x


In [9]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import os
class ImageDataset(Dataset):
    def __init__(self, root_dir, patients_ids, patients_df, transform=None):
        # select the rows of the dataframe that correspond to the patients in the list
        self.patients_df = patients_df[patients_df["patient_id"].isin(patients_ids)]
        self.root_dir = root_dir

    def __len__(self):
        return len(self.patients_df)

    def __getitem__(self, idx):
        # get idx row of the dataframe
        row = self.patients_df.iloc[idx]
        # path = root_dir/patientid_examid_spot_framenumber_score.png
        path = os.path.join(self.root_dir, row["patient_id"] + "_" + row["exam_id"] + "_" + row["spot"] + "_" + row["frame_number"] + "_" + row["score"] + ".png")
        image = Image.open(path)
        # Extract label from the image filename
        label = row["score"]
        # set the image to a tensor
        image = transforms.ToTensor()(image)
        return image, int(label)

In [10]:
# define dataset
import pandas as pd
from pandas_profiling import ProfileReport
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# "patient_id exam_id spot[1-14] frame_number score"
# load patients data in a dataframe from images folder
import glob
images_paths = glob.glob("images/*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[7:-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(str)
images_df["frame_number"] = images_df["frame_number"].astype(str)
images_df["spot"] = images_df["spot"].astype(str)
images_df["patient_id"] = images_df["patient_id"].astype(str)
images_df["exam_id"] = images_df["exam_id"].astype(str)

# profile = ProfileReport(images_df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
# profile.to_file("images_df.html")

# print for each patient the number of images
print(images_df.groupby("patient_id").count()["score"])

patients_ids = set(images_df["patient_id"])
patients_ids = list(patients_ids)
import random
random.shuffle(patients_ids)
train_patients = patients_ids[:int(len(patients_ids)*0.8)]
test_patients = patients_ids[int(len(patients_ids)*0.8):]
print("train patients: ", train_patients, "test patients: ", test_patients)
train_dataset = ImageDataset("images/", train_patients, images_df)
test_dataset = ImageDataset("images/", test_patients, images_df)
print("train dataset length: ", len(train_dataset), "test dataset length: ", len(test_dataset))

patient_id
1017    4747
1045    6999
1047    1151
1048    6407
1050    2760
1051    1239
1052    6260
1066    3505
1067    4260
1068    7008
1069    3385
Name: score, dtype: int64
train patients:  ['1017', '1051', '1066', '1068', '1048', '1045', '1052', '1067'] test patients:  ['1069', '1050', '1047']
train dataset length:  40425 test dataset length:  7296


In [11]:
import numpy as np
from torch import optim
from tqdm import tqdm

def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects/ len(dataloader.dataset)
    return epoch_loss, epoch_acc

def test_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [12]:
data_dir = "images"
k = 5
num_epochs = 10
learning_rate = 0.001

# define the model
model = MIDmodel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# define the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# train the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
    print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
    test_loss, test_acc = test_model(model, test_dataloader, criterion, device)
    print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")

# save the model
torch.save(model.state_dict(), "model.pt")



c:\Users\Davide\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Davide\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1


100%|██████████| 20213/20213 [13:45<00:00, 24.50it/s]


Training Loss: 1.2653 Training Accuracy: 0.4062


100%|██████████| 3648/3648 [00:55<00:00, 65.28it/s]


...

Test Loss: 3.2468 Test Accuracy: 0.2308
Epoch 10


100%|██████████| 20213/20213 [12:54<00:00, 26.10it/s]


Training Loss: 0.0773 Training Accuracy: 0.9725


100%|██████████| 3648/3648 [00:50<00:00, 71.91it/s]

Test Loss: 3.0369 Test Accuracy: 0.3965


In [13]:
# load the model
model = MIDmodel()
model.load_state_dict(torch.load("model.pt"))
model.eval()

MIDmodel(
  (pretrained): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track